### Exercitiul 2

Pentru acest exercitui am implementat 2 variante ale metodei pentru normalizare.

Prima metoda foloseste libraria numpy si a 2-a foloseste scikit-learn.



In [18]:
import numpy as np
def normalize_data(train_data, test_data, type=None):
  if type =='standard':
    mean = np.mean(train_data,axis=0) # calculeaza mediile atributelor din datele de antrenare
    std = np.std(train_data,axis=0) # calculeaza deviatiile standard din datele de antrenare
    train_data -= mean
    train_data/=std
    test_data -= mean
    test_data /=std
  elif type == 'l1':
    train_data = train_data / (np.expand_dims((np.linalg.norm(train_data,ord=1,axis=1)), axis=1) + 1e-6) # normalizarea L1. 1e-6 este adunat pt cazurile in care norma e 0. Iar np.expand_dims e folosit pentru a putea folosi broadcast la impartire
    test_data = test_data / (np.expand_dims((np.linalg.norm(test_data, ord=1,axis=1)), axis=1) + 1e-6)
  elif type == 'l2':
    norm_train = np.expand_dims((np.linalg.norm(train_data,ord=2,axis=1)), axis=1)
    train_data = train_data / (norm_train + 1e-6)
    test_data = test_data / (np.expand_dims((np.linalg.norm(test_data, ord=2,axis=1)), axis=1) +1e-6)
  return train_data, test_data

In [19]:
from sklearn.preprocessing import StandardScaler, Normalizer
def normalize_data_v2(train_data, test_data, type=None):
  if type=='standard':
    scaler = StandardScaler()
    scaler.fit(train_data)
    train_data = scaler.transform(train_data)
    test_data = scaler.transform(test_data)
  elif type=='l2':
    normalizer = Normalizer(norm='l2')
    train_data = normalizer.transform(train_data)
    test_data = normalizer.transform(test_data)
  elif type =='l1':
    normalizer = Normalizer(norm='l1')
    train_data = normalizer.transform(train_data)
    test_data = normalizer.transform(test_data)
  return train_data, test_data

### Exercitiul 3 si 4

In [3]:
import copy
class BagOfWords:

  def __init__(self):
    self.vocab = {}
    self.words = []

  def build_vocabulary(self, data):
    # pentru fiecare cuvant din variabila "data" asociem un id unic,
    # care va reprezenta indexul cuvantului in vectorul de features
    id_word = 0
    for mesaj in data:
      for word in mesaj:
        if word not in self.vocab:
          self.vocab[word]=id_word
          id_word+=1
          self.words.append(word)
  
  def get_features(self, data):
    # transforma textele din var "data" in vectori de dimensiunea vocabularului
    # care contin frecventele cuvintelor.
    feature_matrix = np.zeros((data.shape[0],len(self.vocab)))
    for i, example in enumerate(data):
      no_word = True
      for word in example:
        if word in self.vocab:
          feature_matrix[i,self.vocab[word]] +=1
          no_word = False
      if no_word:
        print(example)
    return feature_matrix

In [6]:
train_data_original = np.load('./data/training_sentences.npy',allow_pickle=True)
train_labels = np.load('./data/training_labels.npy')
test_data_original = np.load('./data/test_sentences.npy',allow_pickle=True)
test_labels = np.load('./data/test_labels.npy')

In [7]:
bow = BagOfWords()
bow.build_vocabulary(train_data_original)
print(len(bow.vocab))

9522


### Exercitiul 5

Urmatoarele 2 celule reprezinta ex5.

In [8]:
train_data_original = bow.get_features(train_data_original)
test_data_original = bow.get_features(test_data_original)

[]
[]
['Ultimately', 'tor', 'motive', 'tui', 'achieve', 'korli']
['22', '146tf150p']
['YO', 'YO', 'YO', 'BYATCH', 'WHASSUP']
['Ryder', 'unsoldnow', 'gibbs']
['Kkcongratulation']
['hanks', 'lotsly']
['645']


In [9]:
train_data, test_data = normalize_data_v2(train_data_original,test_data_original,type='l2')

### Exercitiul 6

Pentru antrenarea modelului folosim functia fit.

Iar cu functia predict obtinem predictiile pentru setul de test.

In [10]:
from sklearn import svm
svm_model = svm.SVC(C=1,kernel= "linear") # define the model
svm_model.fit(train_data, train_labels) # train

SVC(C=1, kernel='linear')

In [11]:
from sklearn.metrics import f1_score, accuracy_score

pred_labels = svm_model.predict(test_data)
print("Accuracy:", accuracy_score(test_labels,pred_labels),"F1-score:", f1_score(test_labels, pred_labels))

Accuracy: 0.9842391304347826 F1-score 0.9409368635437881


Mesajele spam au label-ul 1, iar cele non-spam au label-ul 0. 

Deci cele mai negative cuvinte sunt cele care in functia de decizie au cel mai mare coeficient(>0 bineinteles), iar cele mai pozitive sunt cele cu coeficient mic (<0)

In [17]:
index_sort = np.argsort(svm_model.coef_[0])
bow.words = np.array(bow.words)
print("Most positive words:", bow.words[index_sort[:10]])
print("Most negative words:", bow.words[index_sort[-10:]])



Most positive words: ['&lt#&gt' 'me' 'i' 'Going' 'him' 'Ok' 'I' 'Ill' 'my' 'Im']
Most negative words: ['Text' 'To' 'mobile' 'CALL' 'FREE' 'txt' '&' 'Call' 'Txt' 'STOP']


In [5]:
!unzip ./data_lab5.zip

Archive:  ./data_lab5.zip
  inflating: data/test_labels.npy    
  inflating: data/test_sentences.npy  
  inflating: data/training_labels.npy  
  inflating: data/training_sentences.npy  
